In [ ]:
pip install openai

In [ ]:
pip install langchain

In [ ]:
texts='''Suppose you are a counsellor and you are given some questions and user's reponse 
        to those questions, based on his/her response predict whether he/she is addicted to that substance or not?.
         For the prediction make 3 classes  , 0 for not addicted, 1 for mild addicted, 2 for seveare addicted.
          Give me direct answer 0, 1 or 2 only, I just want integer only in my answer.
           You can give weightage to the response for above questions based on your medical knowledge and data for prediction which affects the most.
           User response to questions are either Yes, No, Sometimes or Prefer not to say'''

In [ ]:
text1='''Suppose you are a counselor and you are given some questions and user's response to those questions.
Based on their response, predict whether they are addicted to a substance or not.
For the prediction, classify the addiction level into 4 classes: 0 for not addicted, 1 for mild addiction, 2 for severe addiction, and 3 for No response.
Please provide an integer answer from 0 to 3 only, corresponding to the predicted addiction level.
You can assign weightage to the responses based on your medical knowledge and data to determine the most influential factors for the prediction.
If user answer all the question as 'Prefer not to say' give output as 3.
The user's response to the questions can be either "Yes," "No," "Sometimes," or "Prefer not to say."
'''

In [ ]:
template2='''Given the questions and responses for addiction prediction:
Questions and Responses:
{user_input}

Predict the user's addiction level by providing the corresponding number:
0 for not addicted, 1 for mild addiction, 2 for severe addiction.
If the user prefers not to answer any question, please ignore that question entirely in the prediction.
If the user prefers not to disclose their addiction level for all questions, output 3 as the answer.
'''

In [ ]:

import os
import openai
import re
from langchain import PromptTemplate, LLMChain
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI, OpenAIChat

OPEN_AI_SECRET = YOUR_API_KEY
os.environ['OPENAI_API_KEY'] = OPEN_AI_SECRET

text1='''Suppose you are a counselor which only output integer value 0,1,2,3 soand you are given some questions and user's answer to those questions.
The user's answer to the questions is either "Yes," "No," "Sometimes," or "Prefer not to say.
Based on their response, predict whether they are addicted to a substance or not.
For the prediction, classify the addiction level into 4 classes: 0 for not addicted, 1 for mild addiction, 2 for severe addiction, and 3 for No response.
Please provide an output as a single integer from 0 to 3 only, corresponding to the predicted addiction level.
You can assign weightage to the responses based on your medical knowledge and data to determine the most influential factors for the prediction.
If user answer all the question as 'Prefer not to say' give output as 3."

For example take 5 questions below
Question 1: A similar one, Do you consume alcohol without being mood-dependent if offered in any situation?
Question 2: Next, Have you tried to quit drinking but couldn't stay abstinent for more than a couple of days?
Question 3: Does the negative or bothersome feeling you experience lead to your alcohol drinking habit?
Question 4: Do you experience alcohol withdrawal symptoms like shaking, anxiety, depression, or morning vomiting when you abstain from drinking for a few days?
Question 5: Last in the section, Do you hiding how much you are drinking from anyone close to you?
Answer1 No
Answer2 No
Answer3 No
Answer4 Prefer not to say
Answer5 No

Your output should be an integer  : 0 in this case.
'''

prefix_messages = [{"role": "system", "content": text1}]

llm = OpenAIChat(model_name='gpt-3.5-turbo', temperature=0.3, prefix_messages=prefix_messages,max_tokens = 100)

template1 = """Take the following question and response of user for an addiction prediction: {user_input}
predict whether he/she is addicted or not based on his response to his/her question just give the output as 0, 1, 2 only it belongs and no analysis, also if you found prefer not to say in entire list of questions given than only  print 3 as output only"""

prompt1 = PromptTemplate(template=template1,  input_variables=["user_input"])

llm_chain = LLMChain(prompt=prompt1, llm=llm)

#Check the code from here
def QnA_store(prompts):
  user_responses = []
  i=1
  for prompt in prompts:
    user_response = input(prompt + " ")   
    user_responses.append("Answer"+str(i) +' '+user_response) # #Storing answer of every question till list ends.Answer1 No
    input_text = "\n".join(prompts + user_responses)
    i+=1
  return input_text  #Collection of all Q set + their respective ans of particular drug
  #When this fxn is called, drug 1's Question will be processed it's answer is stored along with question, once Q end entire Qs with answers processed to LLM for prediction, it is repeated for all drugs

def extract_number(string):
    # Regular Expression
    num = re.search(r'\d+', string)
    if num:
      store= int(num.group())
      if store in [0,1,2,3]:
        return store
      else:
        return 3
    else:
      return 3

prompts_marijuana = [
    "Question 1: Take a near case, Do you consume marijuana without being mood-dependent if offered in any situation?", #Y/N/S
    "Question 2: Cool, Have you experienced decreased appetite?", #Y/N
    "Question 3: Do you feel like no longer participating in activities or events you once enjoyed?",  #Y/N/S
    "Question 4: Do you frequently experience dehydration, nausea, vomiting, or a chronic cough that lasts for more than 8 weeks?", #Y/N/S
    "Question 5: A little dive, Have you found yourself in psychosis (hurting stage) when you regain your senses?"  #Y/N/S
]

prompts_screen=[
    "Question 1: Take a moment,  Do you feel a change in your mood for the worse when you can’t use your device?",  #Y/N/S
    "Question 2: Do you had to work extra for assigned task later to complete your scheduled task?",   #Y/N/S
    "Question 3: Think, Do you frequently uninstall and reinstall an app due to excessive use, without seeing any improvement in your habits?",  #Y/N
    "Question 4: Do you encountered any 2 of these: Fear of missing out, checking phone at night, Phantom Vibrations.",  #Y/N/S
    "Question 5: Well, Do you check people's profiles repeatedly due to anxiety or lie about device/screen use?"  #Y/N
]

prompts_behaviour =[
    "Question 1: Do you have addiction-related thoughts during work or important activities anytimes?",  ##Y/N/S
    "Question 2: Do you undergo risk-taking and increasing risky activities to thrill seek?",   #Y/N/S
    "Question 3: Do you think you need more exposure to feel the highs one you used to feel while doing the activity?", #Y/N/S
    "Question 4: Do you experience cravings for a specific activity, and do you feel shame or embarrassment after engaging in that activity?", #Y/N/S
    "Question 5: Aha, Are you using the activity and justifying it to deal with trauma or poor mental health versus getting professional help?" #Y/N
]
prompts_alcohol=[
    "Question 1: A similar one, Do you consume alcohol without being mood-dependent if offered in any situation?", #Y/N/S
    "Question 2: Next, Have you tried to quit drinking but couldn't stay abstinent for more than a couple of days?", #Y/N
    "Question 3: Does the negative or bothersome feeling you experience lead to your alcohol drinking habit?", ##Y/N
    "Question 4: Do you experience alcohol withdrawal symptoms like shaking, anxiety, depression, or morning vomiting when you abstain from drinking for a few days?", #Y/N/S
    "Question 5: Last in the section, Do you hiding how much you are drinking from anyone close to you?" #Y/N
]
''']
prompts_relationship =[
    "Question 1: Are you being too dependent on your partner?",
    "Question 2: Do you have life outside of the relationship?",
    "Question 3: Do you feel tired, confused, irritable or insecure?",
    "Question 4: Do you think you do not have life outside of the relationship?",
    "Question 5: Do you feel you are experiencing relationship obsessiveness?",
    "Question 6: Take a moment, Do you mostly unable to see the partner’s flaws?"
]
'''
#prompts=[prompts_screen,prompts_behaviour,prompts_marijuana,prompts_alcohol]
prompts=[prompts_marijuana]
#Code for picking set of questions for each addiciton and appending the answers in predictions list in form of 0->not addicted,1->mild addicted,2->seveare addicted 
predictions=[]

for prompt in prompts:
  user_input = QnA_store(prompt)
  pred=llm_chain.run(user_input)
  pred_int= extract_number(pred)
  predictions.append(pred_int)



Question 1: Take a near case, Do you consume marijuana without being mood-dependent if offered in any situation? yes
Question 2: Cool, Have you experienced decreased appetite? no
Question 3: Do you feel like no longer participating in activities or events you once enjoyed? yes
Question 4: Do you frequently experience dehydration, nausea, vomiting, or a chronic cough that lasts for more than 8 weeks? sometimes
Question 5: A little dive, Have you found yourself in psychosis (hurting stage) when you regain your senses? no


In [ ]:
predictions

[2]